In [1]:
import os
import sys

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)

In [2]:
import tensorflow as tf

2023-06-16 02:57:00.869722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 02:57:00.915290: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 02:57:00.916365: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 02:57:01.954553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from data_preprocessing import load_training_dataset

In [4]:
data_path = "../../data/cifar_data/chunks/chunk_1.pickle"

In [5]:
train_ds, test_ds = load_training_dataset(train_dataset_path= data_path)

2023-06-16 02:57:04.363391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-16 02:57:04.363984: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1) # Random brightness
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2) # Random contrast
    return image, label

# Assuming you have loaded your dataset into `train_dataset`
train_dataset = train_ds.map(augment)

In [17]:
train_ds.map(augment)

<_MapDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [14]:
len(train_ds)

63

In [15]:
len(train_dataset)

63

In [16]:
for image, label in train_dataset:
    print(image.shape)
    print(label.shape)

2023-06-16 03:05:29.840968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [7940]
	 [[{{node Placeholder/_1}}]]
2023-06-16 03:05:29.841383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [7940]
	 [[{{node Placeholder/_1}}]]


(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)

In [6]:
from custom_tensorflow_framework import CustomTensorflowFramework

from VGG16 import VGG16

In [7]:
vgg_model = VGG16()

In [8]:
# from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
# centralize_model = TensorflowFramework(vgg_model)
centralize_model = CustomTensorflowFramework(vgg_model)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):

  for images, labels in train_ds:
    train_acc, train_loss = centralize_model.fit(images, labels)

  for test_images, test_labels in test_ds:
    test_acc, test_loss = centralize_model.evaluate(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Accuracy: {train_acc * 100}, '
    f'Loss: {train_loss}, '
    f'Test Accuracy: {test_acc * 100}'
    f'Test Loss: {test_loss}, '
  )

2023-06-16 02:57:07.018972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [7940,32,32,3]
	 [[{{node Placeholder/_0}}]]
2023-06-16 02:57:07.019448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [7940]
	 [[{{node Placeholder/_1}}]]


OperatorNotAllowedInGraphError: in user code:

    File "/home/vtn_ubuntu/ttu/spring23/working_project/AsynFL/training_process/cifar_dataset/client/custom_tensorflow_framework.py", line 36, in train_step  *
        predictions = self.model(images)
    File "/home/vtn_ubuntu/ttu/spring23/working_project/AsynFL/training_process/cifar_dataset/client/VGG16.py", line 10, in __init__  **
        super().__init__(input_features= input_features, output_features= output_features)
    File "/home/vtn_ubuntu/ttu/spring23/working_project/AsynFL/asynfed/client/frameworks/tensorflow/tensorflow_sequential_model.py", line 17, in __init__
        self.create_model(input_features, output_features)
    File "/home/vtn_ubuntu/ttu/spring23/working_project/AsynFL/training_process/cifar_dataset/client/VGG16.py", line 14, in create_model
        self.conv1_1 = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape= input_features)
    File "/home/vtn_ubuntu/miniconda3/envs/asynfed/lib/python3.9/sit

In [15]:
for images, labels in train_ds:
    print(images.shape)
    print(labels.shape)

(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)
(128,)
(128, 32, 32, 3)